<a href="https://colab.research.google.com/github/SameenYounas1/123/blob/main/Medical_Finetuning_With_Qlora_Using_Unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!nvidia-smi

Thu Jan 29 18:21:47 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             32W /   70W |    2652MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [12]:
# Install Unsloth and its dependencies for Colab
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.29" trl peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ym5yt4c7/unsloth_8ea1dcf8804f46529a9972cc2210f73f
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ym5yt4c7/unsloth_8ea1dcf8804f46529a9972cc2210f73f
  Resolved https://github.com/unslothai/unsloth.git to commit e51d3ea2e498fc893770d92ca6727bd113918480
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import pandas as pd
from huggingface_hub import hf_hub_download

# This downloads the MedQuAD CSV from a reliable Hugging Face repository
REPO_ID = "keivalya/MedQuad-MedicalQnADataset"
FILENAME = "medDataset_processed.csv"

# Download and rename it to medquad.csv
dataset_path = hf_hub_download(repo_id=REPO_ID, filename=FILENAME, repo_type="dataset")
df = pd.read_csv(dataset_path)
df.to_csv("medquad.csv", index=False)

print("medquad.csv is now ready in your Colab files!")


medquad.csv is now ready in your Colab files!


In [15]:
# STEP 1: Install Unsloth + dependencies
!pip install -q unsloth
!pip install -q transformers datasets accelerate bitsandbytes peft trl


In [16]:
import torch
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


Torch: 2.9.0+cu126
CUDA available: True
GPU: Tesla T4


In [17]:
import unsloth
print("Unsloth OK")


Unsloth OK


In [18]:
import pandas as pd
from datasets import Dataset

# Unique Prompt Style
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a Clinical Data Specialist. Provide a technical summary from the MedQuAD dataset for the medical inquiry below.

### Question:
{}

### Answer:
{}"""

# 1. Load the CSV
df = pd.read_csv("medquad.csv")

# 2. Match your specific column names (Question and Answer)
# We use .str.strip() just in case there are hidden spaces
df = df[["Question", "Answer"]].dropna()

# 3. Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)

def formatting_prompts_func(examples):
    # Match the capitalized names here as well
    inputs       = examples["Question"]
    outputs      = examples["Answer"]
    texts = []
    for input_text, output_text in zip(inputs, outputs):
        text = alpaca_prompt.format(input_text, output_text)
        texts.append(text)
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True,)
print(f"Dataset formatted for training! 🧬 Rows: {len(df)}")

Map:   0%|          | 0/16407 [00:00<?, ? examples/s]

Dataset formatted for training! 🧬 Rows: 16407


In [20]:

df = df.sample(3000, random_state=42)


df = df[(df["Question"].str.len() < 300) & (df["Answer"].str.len() < 900)]
print("After filter:", len(df))

def format_example(q, a):
    return f"""### Instruction:
You are a helpful medical assistant. Answer clearly and safely.

### Question:
{q}

### Answer:
{a}"""

texts = [format_example(q,a) for q,a in zip(df["Question"], df["Answer"])]

dataset = Dataset.from_dict({"text": texts})
split = dataset.train_test_split(test_size=0.05, seed=42)
train_ds, test_ds = split["train"], split["test"]

print(train_ds, test_ds)
print(train_ds[0]["text"][:400])


After filter: 1544
Dataset({
    features: ['text'],
    num_rows: 1466
}) Dataset({
    features: ['text'],
    num_rows: 78
})
### Instruction:
You are a helpful medical assistant. Answer clearly and safely.

### Question:
What are the symptoms of Hendra Virus Disease (HeV) ?

### Answer:
After an incubation of 9-16 days, infection with Hendra virus can lead to respiratory illness with severe flu-like signs and symptoms. In some cases, illness may progress to encephalitis. 
                
Although infection with Hendra 


In [21]:
from unsloth import FastLanguageModel
import torch

# CHANGE THIS TO 2048
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3.2-3b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
)
print("Model updated to 2048 length! ✅")


==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Model updated to 2048 length! ✅


In [29]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset, # FIXED: Changed from train_ds to dataset
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # 60 steps is perfect for a fast, successful demo
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/1544 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,544 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"
wandb: Using W&B in offline mode.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss
1,2.076000
2,2.125500
3,2.175100
4,2.048300
5,1.947800
6,1.945300
7,1.676400
8,1.609900
9,1.449700
10,1.393500


wandb: WARNING URL not available in offline run


train/epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇███
train/grad_norm,▃▂▂▆▅▅█▃▃▃▁▂▃▁▃▃▃▃▄▂▂▁▁▅▂▂▁▂▃▂▃▃▁▃▁▂▂▄▂▂
train/learning_rate,▂▅███▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁
train/loss,▇██▇▇▅▄▄▃▂▄▃▃▄▄▁▃▂▄▂▃▃▃▃▃▃▂▁▁▁▂▃▂▃▂▂▃▃▃▁
total_flos,1360780091252736.0
train/epoch,0.31088
train/global_step,60
train/grad_norm,0.49025
train/learning_rate,0.0
train/loss,0.877


TrainOutput(global_step=60, training_loss=1.26186470190684, metrics={'train_runtime': 138.9473, 'train_samples_per_second': 3.455, 'train_steps_per_second': 0.432, 'total_flos': 1360780091252736.0, 'train_loss': 1.26186470190684, 'epoch': 0.31088082901554404})

In [30]:
model.save_pretrained("medquad_lora")
tokenizer.save_pretrained("medquad_lora")
print("Saved")

Saved


In [31]:
FastLanguageModel.for_inference(model)

prompt = """### Instruction:
You are a helpful medical assistant. Answer clearly and safely.

### Question:
What is hypertension?

### Answer:
"""

inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
out = model.generate(**inputs, max_new_tokens=120)
print(tokenizer.decode(out[0], skip_special_tokens=True))


### Instruction:
You are a helpful medical assistant. Answer clearly and safely.

### Question:
What is hypertension?

### Answer:
Hypertension is a medical condition in which the blood pressure in the arteries is higher than normal. Blood pressure is the force of blood pushing against artery walls as the heart pumps. When blood pressure is too high, it may damage the heart, brain, kidneys, and eyes. Hypertension is a "silent killer" because it often has no warning signs or symptoms.  High blood pressure is called "the silent killer" because it often has no warning signs or symptoms.  Most people with high blood pressure have no symptoms. However, high blood pressure can cause:  Heart attack or


In [32]:
from unsloth import FastLanguageModel
import torch

FastLanguageModel.for_inference(model)

prompt = """### Instruction:
You are a helpful medical assistant. Answer clearly and safely.

### Question:
What causes glaucoma?

### Answer:
"""

inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
out = model.generate(**inputs, max_new_tokens=160, do_sample=True, temperature=0.7)
print(tokenizer.decode(out[0], skip_special_tokens=True))


### Instruction:
You are a helpful medical assistant. Answer clearly and safely.

### Question:
What causes glaucoma?

### Answer:
Glaucoma is a group of eye diseases that damage the optic nerve. The optic nerve is the part of the eye that carries information from the eye to the brain. The optic nerve fibers are responsible for sending visual information to the brain. As the fibers become damaged, the quality of vision decreases. The damage to the optic nerve causes tunnel vision, which is the loss of peripheral (side) vision. If left untreated, glaucoma can lead to total blindness.  Glaucoma can be caused by several factors. These factors include genetics, age, and other health problems.  Many people with glaucoma have no symptoms. For this reason, it is important to have regular eye examinations. If glaucoma is caught early, it can be treated effectively.  Some eye diseases can


In [33]:
def ask(q):
    prompt = f"""### Instruction:
You are a helpful medical assistant. Answer clearly and safely.

### Question:
{q}

### Answer:
"""
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    out = model.generate(**inputs, max_new_tokens=180, do_sample=True, temperature=0.7)
    print(tokenizer.decode(out[0], skip_special_tokens=True))

ask("What is the difference between type 1 and type 2 diabetes?")
ask("When should someone seek urgent care for high blood pressure?")


### Instruction:
You are a helpful medical assistant. Answer clearly and safely.

### Question:
What is the difference between type 1 and type 2 diabetes?

### Answer:
Type 1 diabetes is an autoimmune disease in which the body's immune system destroys the cells in the pancreas that make insulin. In type 2 diabetes, the body either doesn't produce enough insulin or the insulin that is made doesn't work as it should. These two types of diabetes are not the same, but they are both serious diseases that can cause complications if left untreated. Type 1 diabetes usually begins in childhood or adolescence, but can occur at any age. Type 2 diabetes is more common and usually begins in middle age or later.  The two types of diabetes are similar in that they both require regular monitoring of blood glucose levels and often medications to help control blood glucose levels. Both types of diabetes can lead to serious complications if left untreated. However, the treatment for type 1 diabetes is di

In [34]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1024

# 1) Load base model ONLY (no LoRA)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3.2-3b-bnb-4bit",  #
    max_seq_length=max_seq_length,
    load_in_4bit=True,
)

FastLanguageModel.for_inference(model)

def ask_base(q):
    prompt = f"""### Instruction:
You are a helpful medical assistant. Answer clearly and safely.

### Question:
{q}

### Answer:
"""
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    out = model.generate(**inputs, max_new_tokens=180, do_sample=True, temperature=0.7)
    return tokenizer.decode(out[0], skip_special_tokens=True)

# جرّبي نفس السؤال اللي بدك تقارني فيه
q = "What causes glaucoma?"
base_answer = ask_base(q)
print("=== BEFORE (BASE) ===")
print(base_answer)


==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
=== BEFORE (BASE) ===
### Instruction:
You are a helpful medical assistant. Answer clearly and safely.

### Question:
What causes glaucoma?

### Answer:
There are many causes of glaucoma, but one of the most common causes is elevated eye pressure. If the pressure in your eyes is too high, it can damage your optic nerve, which is the nerve that carries signals from your eyes to your brain. If the pressure is too high for too long, it can cause glaucoma. Other causes of glaucoma include aging, genetics, and eye surgery.


In [35]:
from unsloth import FastLanguageModel

# 2) Attach LoRA structure then load adapter weights
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
)

# Load your saved adapter folder
model.load_adapter("medquad_lora", adapter_name="medquad")
model.set_adapter("medquad")   # تفعيل الأدابتر

FastLanguageModel.for_inference(model)

def ask_lora(q):
    prompt = f"""### Instruction:
You are a helpful medical assistant. Answer clearly and safely.

### Question:
{q}

### Answer:
"""
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    out = model.generate(**inputs, max_new_tokens=180, do_sample=True, temperature=0.7)
    return tokenizer.decode(out[0], skip_special_tokens=True)

lora_answer = ask_lora(q)
print("=== AFTER (LoRA) ===")
print(lora_answer)


=== AFTER (LoRA) ===
### Instruction:
You are a helpful medical assistant. Answer clearly and safely.

### Question:
What causes glaucoma?

### Answer:
The exact cause of glaucoma is unknown. But it is clear that both genetic and environmental factors play a role. Many people with glaucoma have a family history of the disease. In some people, there is a strong genetic component to the development of glaucoma. Other people with glaucoma have no family history of the disease. It is likely that there are many genes involved in glaucoma. These genes work together to determine whether a person will develop glaucoma. Researchers are studying the genetic and environmental factors involved in the development of glaucoma. They hope to better understand the risk factors and develop new treatments. The following risk factors may increase the chances of developing glaucoma:  - Age. As you get older, your risk of developing glaucoma increases.  - Race. The risk of developing glaucoma is higher in p

In [36]:
print("\n\n========================")
print("QUESTION:", q)
print("========================\n")

print("BEFORE (BASE):\n", base_answer[:1200])
print("\n------------------------\n")
print("AFTER (LoRA):\n", lora_answer[:1200])




QUESTION: What causes glaucoma?

BEFORE (BASE):
 ### Instruction:
You are a helpful medical assistant. Answer clearly and safely.

### Question:
What causes glaucoma?

### Answer:
There are many causes of glaucoma, but one of the most common causes is elevated eye pressure. If the pressure in your eyes is too high, it can damage your optic nerve, which is the nerve that carries signals from your eyes to your brain. If the pressure is too high for too long, it can cause glaucoma. Other causes of glaucoma include aging, genetics, and eye surgery.

------------------------

AFTER (LoRA):
 ### Instruction:
You are a helpful medical assistant. Answer clearly and safely.

### Question:
What causes glaucoma?

### Answer:
The exact cause of glaucoma is unknown. But it is clear that both genetic and environmental factors play a role. Many people with glaucoma have a family history of the disease. In some people, there is a strong genetic component to the development of glaucoma. Other people 

In [37]:
question = [
  "What is hypertension?",
  "What are symptoms of diabetes?",
  "When should someone seek urgent care for high blood pressure?",
  "What causes migraine headaches?"
]

for q in question:
    print("\n====================")
    print("Q:", q)
    print("AFTER (LoRA):")
    print(ask_lora(q)[:800])



Q: What is hypertension?
AFTER (LoRA):
### Instruction:
You are a helpful medical assistant. Answer clearly and safely.

### Question:
What is hypertension?

### Answer:
Hypertension is a condition in which the blood pressure in the arteries is higher than it should be. High blood pressure is a major risk factor for heart disease and stroke. It can also cause damage to organs, such as the heart, kidneys, and eyes. Hypertension is also called the silent killer because it often has no symptoms and is not diagnosed until blood pressure is checked.  High blood pressure is usually caused by a combination of factors, including genetics, age, obesity, diet, and physical inactivity.  To diagnose hypertension, a healthcare provider will check your blood pressure. If your blood pressure is higher than normal, you may have hypertension. I

Q: What are symptoms of diabetes?
AFTER (LoRA):
### Instruction:
You are a helpful medical assistant. Answer clearly and safely.

### Question:
What are sympt

In [38]:
import random
random.seed(42)

for i in range(5):
    sample = test_ds[random.randint(0, len(test_ds)-1)]["text"]

    q = sample.split("### Question:")[1].split("### Answer:")[0].strip()

    print("\n====================")
    print("Q:", q)
    print("AFTER (LoRA):")
    print(ask_lora(q)[:700])



Q: How to diagnose Harlequin ichthyosis ?
AFTER (LoRA):
### Instruction:
You are a helpful medical assistant. Answer clearly and safely.

### Question:
How to diagnose Harlequin ichthyosis?

### Answer:
Harlequin ichthyosis is a rare, life-threatening condition that can be diagnosed in the first few days of life. It is a skin disorder caused by a mutation in the ABCA12 gene. This gene provides instructions for making an enzyme that helps the skin cells form a protective barrier. The diagnosis of harlequin ichthyosis is based on the characteristic pattern of skin lesions, which usually begin as large, waxy, yellowish-brown bumps that resemble boils. These lesions may be present at birth or appear within the first few days of life. The lesions beco

Q: What is (are) Severe intellectual disability-progressive spastic diplegia syndrome ?
AFTER (LoRA):
### Instruction:
You are a helpful medical assistant. Answer clearly and safely.

### Question:
What is (are) Severe intellectual disabilit

In [25]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()


In [24]:
# Test the model
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is (are) Glaucoma?", # Question
        "", # Leave answer blank for the AI to fill
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou are a Clinical Data Specialist. Provide a technical summary from the MedQuAD dataset for the medical inquiry below.\n\n### Question:\nWhat is (are) Glaucoma?\n\n### Answer:\nGlaucoma is a group of eye diseases that cause damage to the optic nerve, which is the nerve that carries visual information from the eye to the brain. The most common type of glaucoma is primary open-angle glaucoma, which is the leading cause of blindness in the United States. Glaucoma is a progressive disease, which means that it gets worse over time. There are several types of glaucoma, and each type has its own specific symptoms and treatments. If you have glaucoma, you will need to see an eye doctor regularly to monitor your condition and receive treatment.\n\n### Explanation:\nGlaucoma is']

In [22]:
# 1. Define the "Personality" of your AI (The Prompt)
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a Clinical Data Specialist. Provide a technical summary from the MedQuAD dataset for the medical inquiry below.

### Question:
{}

### Answer:
{}"""

# 2. Format your question
inputs = tokenizer(
[
    alpaca_prompt.format(
        "What are the symptoms of Diabetes?", # You can change this to any medical question!
        "",
    )
], return_tensors = "pt").to("cuda")

# 3. Generate the answer with a professional "word-by-word" look
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

print("--- AI RESPONSE START ---")
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256, repetition_penalty = 1.2)
print("--- AI RESPONSE END ---")

--- AI RESPONSE START ---
<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a Clinical Data Specialist. Provide a technical summary from the MedQuAD dataset for the medical inquiry below.

### Question:
What are the symptoms of Diabetes?

### Answer:
The following table shows how many patients have diabetes and their respective symptom scores based on 10 questions asked by clinicians to assess disease severity:

| Symptom Score | Number of Patients |
|:------------:|:------------------|
|   <font color='green'>0</font>     |          3         |
|      <font color='#FF0000'>&lt;1</font>|           4        |

---

## Explanation

This information can be used in conjunction with other clinical data sources such as electronic health records (EHRs) or patient-reported outcomes (PROs). For instance, one could use this type of analysis alongs

In [41]:
# @title 🩺 Clinical AI Assistant Demo
# @markdown Type a medical question here and press the play button on the left!

User_Question = "What are the complications of untreated Glaucoma?" # @param {type:"string"}

# This uses the prompt format we trained the model on
inputs = tokenizer(
[
    alpaca_prompt.format(
        User_Question,
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

print(f"--- ANALYZING: {User_Question} ---")
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512, repetition_penalty = 1.2)

--- ANALYZING: What are the complications of untreated Glaucoma? ---
<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a Clinical Data Specialist. Provide a technical summary from the MedQuAD dataset for the medical inquiry below.

### Question:
What are the complications of untreated Glaucoma?

### Answer:
Glaucoma can lead to severe vision loss or blindness if left untreated and may require surgery. However, there are medications available today that help prevent glaucomatous damage by lowering intraocular pressure (IOP). These include prostaglandin analogs such as bimatoprost, travoprost, latanoprost, and tafluprost; alpha-adrenergic agonists like brimonidine tartrate and apraclonidine hydrochloride; carbonic anhydrase inhibitors like dorzolamide hydrochloride and brinzolamide; and beta-blockers like timolol maleate. In addition to th

In [39]:
# 1. Create a zip file containing the model folder and the dataset
# !zip -r [name_of_zip] [folder_to_zip] [other_files]
!zip -r Medical_AI_Project.zip drive/MyDrive/medical_model_lora medquad.csv

print("Zip file created successfully! Look for Medical_AI_Project.zip in the left sidebar.")

	zip warning: name not matched: drive/MyDrive/medical_model_lora
  adding: medquad.csv (deflated 78%)
Zip file created successfully! Look for Medical_AI_Project.zip in the left sidebar.


In [40]:
# 1. Create the zip folder containing ALL components
# This includes: your trained model, the dataset, and any output files
!zip -r Medical_AI_Complete_Project.zip drive/MyDrive/medical_model_lora medquad.csv

print("Full project zipped! Now download 'Medical_AI_Complete_Project.zip' from the sidebar.")

	zip warning: name not matched: drive/MyDrive/medical_model_lora
  adding: medquad.csv (deflated 78%)
Full project zipped! Now download 'Medical_AI_Complete_Project.zip' from the sidebar.
